In [ ]:
%pylab inline
from parcels import FieldSet, Field, ParticleSet, JITParticle, AdvectionRK4, ErrorCode, Variable
import matplotlib.patches as mpatches
import cartopy
from datetime import timedelta as delta
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import xarray as xr
from os import environ
import matplotlib.animation as animation
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

In [ ]:
# exps = ['bwd', 'bwd_wstokes']
# exps = ['americas', 'americas_wstokes']
exps = ['bwd']

titles = {'americas': 'Currents only from Americas', 'americas_wstokes': 'Currents and waves from Americas', 
         'bwd': 'Currents only from Galapagos', 'bwd_wstokes': 'Currents and waves from Galapagos'}
colors = {'americas': 'C8', 'americas_wstokes': 'C9'}

countrylats = [-60, -18, -3, 1.5, 7.3, 8.1, 11.0, 12.8, 13.7, 14.6, 32.5]
countrynames = ['Chile', 'Peru', 'Ecuador', 'Colombia', 'PA', 'CR', 'NI', 'SV', 'GT', 'Mexico']

In [ ]:
galapagos_extent = [-91.8, -89, -1.4, 0.7]
environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

def load_data(exp):
    T = {}
    pfile = xr.open_dataset('/scratch/evansebille/galapagosparticles_%s.nc' %exp, decode_cf=True)
    T['lon'] = np.ma.filled(pfile.variables['lon'], np.nan)
    T['lat'] = np.ma.filled(pfile.variables['lat'], np.nan)
    T['time'] = np.ma.filled(pfile.variables['time'], np.nan)
    T['age'] = np.ma.filled(pfile.variables['age'], np.nan)

    if 'bwd' not in exp:
        T['visitedgalapagos'] = np.ma.filled(pfile.variables['visitedgalapagos'], np.nan)
    else:
        T['visitedgalapagos'] = np.ones(T['lat'].shape)
    pfile.close()

    if 'bwd' not in exp:
        T['visitedtime'] = np.nan*np.zeros(T['visitedgalapagos'].shape[0])
        for p in range(T['visitedgalapagos'].shape[0]):
            if np.any(T['visitedgalapagos'][p, :] == 1):
                I = np.where(T['visitedgalapagos'][p, :] == 1)[0][0]
                T['visitedtime'][p] = T['age'][p, I]/86400.
                T['visitedgalapagos'][p, :] = 1
        T['nvisited'] = len(np.where(T['visitedgalapagos'][:, 0]==1)[0])

    T['nsites'] = np.unique(T['lat'][:, 0]).shape[0]
    T['startsite'] = np.nan*np.zeros(T['lat'].shape[0])
    T['startcountry'] = np.nan*np.zeros(T['lat'].shape[0])
    for s in range(T['nsites']):
        I = np.where(T['lat'][:, 0] == T['lat'][s, 0])
        T['startsite'][I] = s
        if 'americas'in exp:
            C = np.where(T['lat'][s, 0] > countrylats)[0]
            T['startcountry'][I] = C[-1]

    return T
    
D = {}
for exp in exps:
    D[exp] = load_data(exp)

In [ ]:
from matplotlib import colors
pylab.rcParams['figure.figsize'] = (8, 10)

projection = cartopy.crs.PlateCarree(central_longitude=180)
fig, ax = plt.subplots(len(D), 1,subplot_kw={'projection': projection})
if len(D) == 1:
    ax = [ax]

for i, exp in enumerate(exps):
    T = D[exp]
    ax[i].coastlines()
    ax[i].add_feature(cartopy.feature.LAND)

    if 'americas'in exp:
        bins = [np.arange(-260, -60, 1), np.arange(-60, 45, 1)]
        recfc = None
    elif exp == 'china':
        bins = [np.arange(90, 200, 1), np.arange(-20, 50, 1)]
        recfc = None
    elif 'bwd' in exp:
        bins = [np.arange(-200, -60, 1), np.arange(-50, 45, 1)]
#         bins = [np.arange(-300, -60, 1), np.arange(-80, 45, 1)]
        recfc = 'red'

    ax[i].add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]], width=galapagos_extent[1]-galapagos_extent[0], 
                                    height=galapagos_extent[3]-galapagos_extent[2], facecolor=recfc, alpha=1, linewidth=1,
                                    edgecolor='k', transform=cartopy.crs.Geodetic()))

    H, xe, ye = np.histogram2d(T['lon'][~np.isnan(T['lon'])], T['lat'][~np.isnan(T['lat'])], bins=bins)
    xb = (xe[1:] + xe[:-1])/2
    yb = (ye[1:] + ye[:-1])/2

    levels = [10**x for x in range(0, 9)]
    co = ax[i].contourf(xb, yb, H.T, levels=levels, transform=cartopy.crs.PlateCarree(), norm=colors.LogNorm())
    ax[i].set_extent([bins[0][0], bins[0][-1], bins[1][0], bins[1][-1]], crs=cartopy.crs.PlateCarree())
    ax[i].set_title(titles[exp])

cbaxes = fig.add_axes([0.1, 0.05, 0.8, 0.03]) 
cb = plt.colorbar(co, orientation="horizontal", cax = cbaxes)

plt.show()
# savefig('histograms.pdf')

In [ ]:
pylab.rcParams['figure.figsize'] = (20, 7)

years = range(2000, 2011)

fig, ax = plt.subplots(1, len(D))
if len(D) == 1:
    ax = [ax]

for i, exp in enumerate(exps):
    T = D[exp]

    yrs = T['time'][:, 0].astype('datetime64[Y]').astype('int')+1970
    countriestoplot = [1, 2, 3]
    fracperyear = np.nan*np.zeros((len(countriestoplot), len(years)))
    for c, country in enumerate(countriestoplot):
        for y in range(len(years)):
            I = np.where(np.logical_and(yrs == years[y], T['startcountry'] == country))[0]
            J = np.where(np.isfinite(T['visitedtime'][I]))[0]
            if len(I) > 0:
                fracperyear[c, y] = 100.0*len(J)/len(I)

    lineObjects = ax[i].plot([y+0.5 for y in years], fracperyear.T, 'o-')
    ax[i].legend(iter(lineObjects), [T['countrynames'][c] for c in countriestoplot])
    ax[i].set_xticks(years)
    ax[i].set_xlabel('Year')
    ax[i].set_ylabel('Fraction of particles that pass through Galapagos [%]')
    ax[i].set_title(titles[exp])

    yl = [0, 12]
    for elnino in [2003, 2005, 2007]:
        ax[i].add_collection(PatchCollection([Rectangle((elnino-1./12, 0), 3./12, yl[1])], 
                             facecolor='r', alpha=0.2, edgecolor=None))
        ax[i].text(elnino+0.5/12., yl[1]-0.5, 'EL', horizontalalignment='center')

    for lanina in [2000, 2001, 2008, 2009, 2010]:
        ax[i].add_collection(PatchCollection([Rectangle((lanina-1./12, 0), 3./12, yl[1])], 
                             facecolor='b', alpha=0.2, edgecolor=None))
        ax[i].text(lanina+0.5/12., yl[1]-0.5, 'LN', horizontalalignment='center')
    ax[i].set_ylim(yl)
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 6)

for i, exp in enumerate(exps):
    T = D[exp]
    if np.any(np.isfinite(T['visitedtime'])):
        plt.hist(T['visitedtime'][np.isfinite(T['visitedtime'])], np.arange(0, 365*2, 10), 
                 color=colors[exp], alpha=0.5, label=titles[exp])
plt.gca().set_xlim([0, 365*2])
plt.legend()
plt.xlabel('Time to first pass through Galapagos [days]')
plt.ylabel('Number of particles')
plt.show()

In [ ]:
for i, exp in enumerate(exps):
    T = D[exp]

    pct_visitgalapagos = np.zeros(T['nsites'])
    startlat = np.zeros(T['nsites'])
    for s in range(T['nsites']):
        I = np.where(T['startsite']==s)[0]
        nvisit = len(np.where(T['visitedgalapagos'][I,0]==1)[0])
        pct_visitgalapagos[s] = 100*nvisit/len(I)
        startlat[s] = T['lat'][I[0], 0]
    plt.plot(startlat, pct_visitgalapagos,'.-', color=colors[exp], label=titles[exp])

plt.xlabel('Starting latitude [degrees N]')
plt.ylabel('Fraction of particles passing through Galapagos [%]')
plt.legend()
ylim = [0, 25]
xlim = [-25, 25]
for s in range(1, len(countrylats)):
    plt.plot([countrylats[s], countrylats[s]], ylim, 'k:')
    xmin = max(xlim[0], countrylats[s-1])
    xmax = min(xlim[1], countrylats[s])
    plt.text((xmin+xmax)/2, ylim[1]-5, countrynames[s-1], horizontalalignment='center')
plt.xlim(xlim)
plt.ylim(ylim)
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 7)
projection = cartopy.crs.PlateCarree(central_longitude=180)

for i, exp in enumerate(exps):
    T = D[exp]
    fig, ax = plt.subplots(1, 1,subplot_kw={'projection': projection})
    ax.coastlines()
    ax.add_feature(cartopy.feature.LAND)

    ax.add_patch(mpatches.Rectangle(xy=[galapagos_extent[0], galapagos_extent[2]],
                                width=galapagos_extent[1]-galapagos_extent[0], 
                                height=galapagos_extent[3]-galapagos_extent[2], facecolor='red', alpha=0.2,
                                transform=cartopy.crs.PlateCarree()))

    if 'americas'in exp:
        axextent = [-260, -60, -40, 35]
    elif exp == 'china':
        axextent = [-90, 120, -20, 50] 
    elif 'bwd' in exp:
        axextent = [-300, -60, -80, 45] 
    plottimes = np.arange(np.min(T['time']), np.max(T['time']), dtype='datetime64[5D]')

    b = np.where(np.logical_and(T['time'] >= plottimes[0], T['time'] < plottimes[1]))
    scat = ax.scatter(T['lon'][b], T['lat'][b], s=3, c=T['visitedgalapagos'][b], 
                      transform=cartopy.crs.Geodetic(), cmap="bwr")
    ax.set_extent(axextent, crs=cartopy.crs.PlateCarree())
    scat.set_clim(0, 1)

    ttl = ax.set_title(titles[exp] + ' on ' + str(plottimes[0]) + ' (red particles pass through Galapagos)')
    frames = np.arange(0, len(plottimes)-1)

    def animate(t):
        b = np.where(np.logical_and(T['time'] >= plottimes[t], T['time'] < plottimes[t+1]))
        scat.set_offsets(np.vstack((T['lon'][b], T['lat'][b])).transpose())
        scat.set_array(T['visitedgalapagos'][b])
        ttl.set_text(titles[exp] + ' on ' + str(plottimes[t]) + ' (red particles pass through Galapagos)')
        return scat,

    ax.set_extent(axextent, crs=cartopy.crs.PlateCarree())
    anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100, blit=True)
    anim
    anim.save('galapagosparticles_%s.gif' % exp, writer='imagemagick', fps=10)